In [2]:
import json
import re

In [64]:
data=[]
# for line in open('C:/Users/MoonJu/Documents/분석 프로젝트/Document summary/extractive_test_v2.jsonl', 'r', encoding='utf-8'):
#     data.append(json.loads(line))

# for line in open('C:/Users/Playdata/Downloads/extractive_test_v2.jsonl', 'r', encoding='utf-8'):
#     data.append(json.loads(line))

for line in open('D:/BigData/document_extraction_data/extractive_test_v2.jsonl', 'r', encoding='utf-8'):
    data.append(json.loads(line))

In [68]:
# 전처리
def CleanText(readData, Num=False, Eng=False):
# 번역
    
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ ]+', '', readData)

    return text.strip(' ')

In [69]:
# 사용할 데이터 정리
clean_article = []
ori_article = []
id_list = []
for i in range(len(data)):
    s1 = []
    s2 = []
    id_list.append(data[i]['id'])
    for j in range(len(data[i]['article_original'])):
        s1.append(data[i]['article_original'][j])
        s2.append(CleanText(data[i]['article_original'][j]))
    ori_article.append(s1)
    clean_article.append(s2)

In [23]:
from collections import Counter
from collections import defaultdict
from scipy.sparse import csr_matrix
import numpy as np
from sklearn.preprocessing import normalize
import math


In [24]:
#키워드 추출

In [25]:
def scan_vocabulary(sents, tokenize, min_count=2):
    counter = Counter(w for sent in sents for w in tokenize(sent))
    counter = {w:c for w,c in counter.items() if c >= min_count}
    idx_to_vocab = [w for w, _ in sorted(counter.items(), key=lambda x:-x[1])]
    vocab_to_idx = {vocab:idx for idx, vocab in enumerate(idx_to_vocab)}
    return idx_to_vocab, vocab_to_idx

In [26]:
def cooccurrence(tokens, vocab_to_idx, window=2, min_cooccurrence=2):
    counter = defaultdict(int)
    for s, tokens_i in enumerate(tokens):
        vocabs = [vocab_to_idx[w] for w in tokens_i if w in vocab_to_idx]
        n = len(vocabs)
        for i, v in enumerate(vocabs):
            if window <= 0:
                b, e = 0, n
            else:
                b = max(0, i - window)
                e = min(i + window, n)
            for j in range(b, e):
                if i == j:
                    continue
                counter[(v, vocabs[j])] += 1
                counter[(vocabs[j], v)] += 1
    counter = {k:v for k,v in counter.items() if v >= min_cooccurrence}
    n_vocabs = len(vocab_to_idx)
    return dict_to_mat(counter, n_vocabs, n_vocabs)

In [27]:
def dict_to_mat(d, n_rows, n_cols):
    rows, cols, data = [], [], []
    for (i, j), v in d.items():
        rows.append(i)
        cols.append(j)
        data.append(v)
    return csr_matrix((data, (rows, cols)), shape=(n_rows, n_cols))

In [28]:
def word_graph(sents, tokenize=None, min_count=2, window=2, min_cooccurrence=2):
    idx_to_vocab, vocab_to_idx = scan_vocabulary(sents, tokenize, min_count)
    tokens = [tokenize(sent) for sent in sents]
    g = cooccurrence(tokens, vocab_to_idx, window, min_cooccurrence, verbose)
    return g, idx_to_vocab

In [29]:
def pagerank(x, df=0.85, max_iter=30):
    assert 0 < df < 1

    # initialize
    A = normalize(x, axis=0, norm='l1')
    R = np.ones(A.shape[0]).reshape(-1,1)
    bias = (1 - df) * np.ones(A.shape[0]).reshape(-1,1)

    # iteration
    for _ in range(max_iter):
        R = df * (A * R) + bias

    return R

In [30]:
def textrank_keyword(sents, tokenize, min_count, window, min_cooccurrence, df=0.85, max_iter=30, topk=30):
    g, idx_to_vocab = word_graph(sents, tokenize, min_count, window, min_cooccurrence)
    R = pagerank(g, df, max_iter).reshape(-1)
    idxs = R.argsort()[-topk:]
    keywords = [(idx_to_vocab[idx], R[idx]) for idx in reversed(idxs)]
    return keywords

In [31]:
# 핵심 문자 추출

In [32]:
def sent_graph(sents, tokenize, similarity, min_count=2, min_sim=0.3):
    _, vocab_to_idx = scan_vocabulary(sents, tokenize, min_count)

    tokens = [[w for w in tokenize(sent) if w in vocab_to_idx] for sent in sents]
    rows, cols, data = [], [], []
    n_sents = len(tokens)
    for i, tokens_i in enumerate(tokens):
        for j, tokens_j in enumerate(tokens):
            if i >= j:
                continue
#             print(tokens_i, tokens_j)
            sim = similarity(tokens_i, tokens_j)
#             print(sim)
            if sim < min_sim:
                continue
            rows.append(i)
            cols.append(j)
            data.append(sim)
    return csr_matrix((data, (rows, cols)), shape=(n_sents, n_sents))

In [33]:
def textrank_sent_sim(s1, s2):
    n1 = len(s1)
    n2 = len(s2)
    if (n1 <= 1) or (n2 <= 1):
        return 0
    common = len(set(s1).intersection(set(s2)))
    base = math.log(n1) + math.log(n2)
    return common / base

In [34]:
def cosine_sent_sim(s1, s2):
    if (not s1) or (not s2):
        return 0

    s1 = Counter(s1)
    s2 = Counter(s2)
    norm1 = math.sqrt(sum(v ** 2 for v in s1.values()))
    norm2 = math.sqrt(sum(v ** 2 for v in s2.values()))
    prod = 0
    for k, v in s1.items():
        prod += v * s2.get(k, 0)
    return prod / (norm1 * norm2)

In [35]:
def textrank_keysentence(sents, tokenize, min_count, min_sim, similarity, df=0.85, max_iter=30, topk=5):
    g = sent_graph(sents, tokenize,  similarity ,min_count, min_sim )
    R = pagerank(g, df, max_iter).reshape(-1)
    idxs = R.argsort()[-topk:]
    keysents = [(idx, R[idx], sents[idx]) for idx in reversed(idxs)]
    return keysents

In [36]:
# 토크나이저
from konlpy.tag import Komoran

komoran = Komoran()
def komoran_tokenize(sent):
    words = komoran.pos(sent, join=True)
    words = [w for w in words if ('/NN' in w or '/XR' in w or '/VA' in w or '/VV' in w)]
    return words

In [53]:
# df['article_original'][0]

idx_list = []
for sents in clean_article:
#     news_li = sents['article_original'].split()
    keysents = textrank_keysentence(sents, 
                                    komoran_tokenize,
                                    min_count= 2, 
                                    min_sim = 0.3, 
                                    similarity = textrank_sent_sim,
                                    topk=3
                                   )
    idx = []
    for sent_idx, rank, sent in keysents:
        idx.append(sent_idx)
    idx_list.append(idx)

In [55]:
result = []
for i in range(len(idx_list)) :
    ori_str = []
    for j in range(3) :
        ori_str.append(ori_article[i][idx_list[i][j]])
    result.append('\n'.join(ori_str))

In [56]:
import pandas as pd
result = pd.DataFrame(
                {'id' : id_list,
                 'summary' : result},
                )

In [ ]:
# 결과 저장
# result.to_csv('C:/Users/MoonJu/Documents/분석 프로젝트/Document summary/extractive_submission_7.csv', encoding="utf-8", index=False)
result.to_csv('D:/BigData/document_extraction_data/extractive_submission_8.csv', encoding="utf-8", index=False)